In [1]:
# importing libraries 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, InputLayer
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# mounting drive
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# reading in the training CSV file
datatraining = pd.read_csv('/content/gdrive/MyDrive/Data/x_train.csv')
datatraining

,Unnamed: 0,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,...,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,729561,0.2,0.6,0.4,1.0,0.8,0.8,0.2,1.0,0.2,0.8,0.6,0.8,0.8,0.6,0.8,0.6,0.4,0.2,1.0,0.6,0.4,0.8,0.4,1.0,0.2,0.4,0.4,0.8,0.8,1.0,1.0,0.2,1.0,0.2,0.6,0.2,1.0,0.8,1.0,...,0.002354,0.007533,0.013656,0.000209,0.000031,0.001510,0.000145,0.000566,0.006730,0.000394,0.004639,0.120108,0.000088,0.009391,0.003659,0.000036,0.000752,0.009806,0.000096,0.000017,0.073986,0.000213,0.002165,0.000036,0.000826,0.025482,0.000175,0.000315,0.000289,0.010641,2018-05-03 06:56:37,1366.0,768.0,20.0,261.0,16,1,AU,-37.974,145.256
1,503683,0.4,0.4,0.8,0.4,0.8,0.4,0.2,0.4,0.4,0.8,0.8,0.8,1.0,0.4,0.8,0.8,0.8,0.8,0.8,0.8,0.2,1.0,0.2,1.0,0.4,1.0,0.4,0.8,1.0,0.8,0.8,0.8,1.0,0.4,0.2,0.8,0.4,0.6,0.8,...,0.002354,0.007535,0.013718,0.000206,0.000031,0.001494,0.000159,0.000567,0.006669,0.000365,0.004631,0.120109,0.000088,0.009391,0.003654,0.000038,0.000760,0.009757,0.000094,0.000016,0.073980,0.000216,0.002227,0.000040,0.005533,0.025490,0.000164,0.000307,0.000282,0.010641,2017-10-03 15:20:06,1280.0,800.0,14.0,1002.0,13,1,US,32.7368,-117.2402
2,219163,1.0,0.4,0.4,0.4,0.8,1.0,0.6,1.0,0.6,0.2,0.4,0.8,0.4,0.4,0.6,0.8,0.8,1.0,0.6,0.8,0.4,0.6,0.4,0.8,0.4,0.0,0.0,0.6,0.6,1.0,0.8,0.2,0.6,0.2,0.4,0.2,0.4,0.8,0.2,...,0.002353,0.007530,0.013618,0.000201,0.000029,0.001483,0.000131,0.000560,0.006657,0.000359,0.004634,0.120096,0.000087,0.009388,0.003652,0.000035,0.000751,0.009753,0.000099,0.000017,0.073985,0.000210,0.002149,0.000036,0.000839,0.025478,0.000161,0.000302,0.000279,0.010640,2016-09-27 08:30:59,375.0,667.0,5.0,117.0,21,24,US,25.7743,-80.1937
3,725015,1.0,0.2,0.8,0.2,1.0,0.2,0.8,0.2,1.0,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.6,0.4,0.4,0.6,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,1.0,1.0,0.8,0.8,0.6,0.8,0.4,1.0,0.6,0.6,0.4,...,0.002351,0.007527,0.013620,0.000207,0.000029,0.001504,0.000156,0.000561,0.006664,0.000368,0.004630,0.120109,0.000088,0.009396,0.003654,0.000038,0.000762,0.009765,0.000096,0.000017,0.073981,0.000212,0.002170,0.000036,0.000800,0.025480,0.000167,0.000322,0.000298,0.010638,2018-04-30 11:25:20,360.0,640.0,7.0,176.0,5,1,CA,45.6,-73.6167
4,427907,0.4,0.8,0.6,0.6,0.6,0.4,0.2,0.8,0.8,0.8,1.0,0.8,0.8,0.4,0.8,0.8,0.8,0.6,0.6,0.6,0.2,0.8,0.2,1.0,0.4,1.0,0.2,0.8,1.0,0.6,0.8,0.8,1.0,0.4,0.4,0.6,0.8,0.4,1.0,...,0.002351,0.007556,0.013632,0.000209,0.000032,0.001496,0.000146,0.000564,0.006663,0.000375,0.004635,0.120108,0.000089,0.009399,0.003654,0.000036,0.000752,0.009773,0.000104,0.000017,0.074012,0.000219,0.002164,0.000037,0.000812,0.025499,0.000181,0.000318,0.000282,0.010643,2017-05-31 18:58:40,320.0,568.0,13.0,411.0,7,39,US,47.4235,-120.3103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710733,259178,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,1.0,0.6,1.0,0.2,0.2,1.0,0.8,0.8,1.0,1.0,0.4,0.6,0.2,1.0,0.4,1.0,0.6,0.6,1.0,0.2,0.2,0.8,0.8,1.0,0.2,0.8,0.8,0.8,0.2,...,0.002360,0.007532,0.013694,0.000208,0.000033,0.001487,0.000140,0.000562,0.006687,0.000385,0.004627,0.120144,0.000092,0.009398,0.003654,0.000037,0.000773,0.009764,0.000100,0.000018,0.073983,0.000226,0.002180,0.000037,0.000815,0.025501,0.000166,0.000383,0.000382,0.010646,2016-11-06 18:44:40,360.0,640.0,6.0,364.0,12,1,US,38.0,-97.0
710734,365838,0.8,0.2,0.8,0.6,0.8,0.0,1.0,0.4,0.8,0.4,0.6,0.8,0.6,0.0,0.6,0.6,0.0,0.0,0.6,0.6

In [4]:
# dropping the empty entries 
datatraining = datatraining.dropna()

In [5]:
# getting rid of unneeded columns 
datatraining = datatraining.drop(['EXT1_E', 'EXT2_E', 'EXT3_E', 'EXT4_E', 'EXT5_E', 'EXT6_E', 'EXT7_E', 'EXT8_E', 'EXT9_E', 'EXT9_E', 'EXT10_E'], axis = 1)
datatraining = datatraining.drop(['EST1_E', 'EST2_E', 'EST3_E', 'EST4_E', 'EST5_E', 'EST6_E', 'EST7_E', 'EST8_E', 'EST9_E', 'EST9_E', 'EST10_E'], axis = 1)
datatraining = datatraining.drop(['AGR1_E', 'AGR2_E', 'AGR3_E', 'AGR4_E', 'AGR5_E', 'AGR6_E', 'AGR7_E', 'AGR8_E', 'AGR9_E', 'AGR9_E', 'AGR10_E'], axis = 1)
datatraining = datatraining.drop(['CSN1_E', 'CSN2_E', 'CSN3_E', 'CSN4_E', 'CSN5_E', 'CSN6_E', 'CSN7_E', 'CSN8_E', 'CSN9_E', 'CSN9_E', 'CSN10_E'], axis = 1)
datatraining = datatraining.drop(['OPN1_E', 'OPN2_E', 'OPN3_E', 'OPN4_E', 'OPN5_E', 'OPN6_E', 'OPN7_E', 'OPN8_E', 'OPN9_E', 'OPN9_E', 'OPN10_E'], axis = 1)
datatraining = datatraining.drop(['dateload', 'screenw', 'screenh', 'introelapse', 'testelapse', 'endelapse', 'IPC', 'country', 'lat_appx_lots_of_err', 'long_appx_lots_of_err'], axis = 1)
datatraining = datatraining.drop(['Unnamed: 0'], axis = 1)

datatraining

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,0.2,0.6,0.4,1.0,0.8,0.8,0.2,1.0,0.2,0.8,0.6,0.8,0.8,0.6,0.8,0.6,0.4,0.2,1.0,0.6,0.4,0.8,0.4,1.0,0.2,0.4,0.4,0.8,0.8,1.0,1.0,0.2,1.0,0.2,0.6,0.2,1.0,0.8,1.0,1.0,0.8,0.2,0.8,0.2,0.8,0.2,0.8,1.0,1.0,1.0
1,0.4,0.4,0.8,0.4,0.8,0.4,0.2,0.4,0.4,0.8,0.8,0.8,1.0,0.4,0.8,0.8,0.8,0.8,0.8,0.8,0.2,1.0,0.2,1.0,0.4,1.0,0.4,0.8,1.0,0.8,0.8,0.8,1.0,0.4,0.2,0.8,0.4,0.6,0.8,0.4,1.0,0.4,0.8,0.6,0.8,0.4,0.8,0.4,0.8,0.8
2,1.0,0.4,0.4,0.4,0.8,1.0,0.6,1.0,0.6,0.2,0.4,0.8,0.4,0.4,0.6,0.8,0.8,1.0,0.6,0.8,0.4,0.6,0.4,0.8,0.4,0.0,0.0,0.6,0.6,1.0,0.8,0.2,0.6,0.2,0.4,0.2,0.4,0.8,0.2,0.8,0.8,0.6,0.0,1.0,0.4,0.6,0.4,0.4,1.0,0.4
3,1.0,0.2,0.8,0.2,1.0,0.2,0.8,0.2,1.0,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.6,0.4,0.4,0.6,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,1.0,1.0,0.8,0.8,0.6,0.8,0.4,1.0,0.6,0.6,0.4,0.8,1.0,0.2,0.8,0.2,0.8,0.2,0.8,0.8,0.8,1.0
4,0.4,0.8,0.6,0.6,0.6,0.4,0.2,0.8,0.8,0.8,1.0,0.8,0.8,0.4,0.8,0.8,0.8,0.6,0.6,0.6,0.2,0.8,0.2,1.0,0.4,1.0,0.2,0.8,1.0,0.6,0.8,0.8,1.0,0.4,0.4,0.6,0.8,0.4,1.0,1.0,0.6,0.4,0.6,0.4,0.8,0.4,1.0,0.6,0.8,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710733,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,0.2,1.0,1.0,0.6,1.0,0.2,0.2,1.0,0.8,0.8,1.0,1.0,0.4,0.6,0.2,1.0,0.4,1.0,0.6,0.6,1.0,0.2,0.2,0.8,0.8,1.0,0.2,0.8,0.8,0.8,0.2,0.4,1.0,0.4,1.0,0.8,0.8,0.2,1.0,0.8,0.6,0.6
710734,0.8,0.2,0.8,0.6,0.8,0.0,1.0,0.4,0.8,0.4,0.6,0.8,0.6,0.0,0.6,0.6,0.0,0.0,0.6,0.6,0.8,0.4,0.2,0.8,0.0,0.4,0.8,0.8,0.8,0.8,1.0,0.2,0.6,0.4,1.0,0.2,1.0,0.6,0.8,0.8,1.0,0.2,0.6,0.8,0.8,0.4,0.6,0.6,0.6,0.8
710735,0.8,0.4,0.6,0.4,1.0,0.2,0.8,0.4,1.0,0.4,0.4,0.4,0.6,0.8,0.4,0.4,0.4,0.2,0.6,0.4,0.6,0.6,0.4,0.6,0.6,0.4,0.4,0.8,0.6,0.8,0.8,0.6,1.0,0.4,0.6,0.2,0.6,0.4,0.8,1.0,1.0,0.2,0.4,0.4,0.8,0.6,1.0,1.0,0.8,1.0
710736,0.2,0.8,0.6,0.6,0.4,0.8,0.4,0.8,0.4,1.0,0.4,0.4,0.8,0.4,0.4,0.4,0.6,0.6,0.4,0.8,0.4,0.4,0.4,0.4,0.8,0.4,0.8,0.6,0.8,0.8,0.8,0.4,1.0,0.4,0.4,0.4,0.8,0.6,0.6,0.8,0.6,0.4,0.4,0.6,0.6,0.8,0.8,0.6,0.8,0.4


In [6]:
# converting to float
datatraining = datatraining.astype('float')

In [7]:
# putting data in a matrix
datatraining = np.asarray(datatraining)
datatraining.shape

(708431, 50)

In [8]:
# reading in the testing CSV file 
datatesting = pd.read_csv('/content/gdrive/MyDrive/Data/x_test.csv')
datatesting

,Unnamed: 0,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,...,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,564534,0.6,0.6,0.6,0.4,0.6,0.4,0.4,0.6,0.6,1.0,0.6,0.8,0.6,0.8,0.4,0.4,0.4,0.4,0.8,0.4,0.2,0.8,0.6,0.8,0.4,1.0,0.4,0.6,0.8,0.8,0.8,0.6,0.8,0.4,0.8,0.4,0.6,0.2,0.8,...,0.002354,0.007531,0.013639,0.000206,0.000031,0.001511,0.000140,0.000562,0.006685,0.000373,0.004632,0.120109,0.000091,0.009395,0.003656,0.000038,0.000779,0.009767,0.000103,0.000017,0.073982,0.000217,0.002160,0.000036,0.000813,0.025487,0.000167,0.000322,0.000295,0.010641,2018-01-01 07:51:30,414.0,736.0,4.0,228.0,13,1,US,42.8706,-71.9518
1,609728,0.6,0.4,0.8,0.8,0.8,0.2,0.6,1.0,0.4,0.8,0.4,0.8,0.4,0.4,0.4,0.4,0.8,0.6,0.6,0.8,0.2,1.0,0.6,1.0,0.2,0.8,0.2,0.8,0.8,1.0,0.4,0.8,0.8,0.6,0.4,0.4,0.8,0.8,0.4,...,0.002359,0.007529,0.013646,0.000206,0.000030,0.001497,0.000136,0.000561,0.006726,0.000366,0.004630,0.120115,0.000088,0.009395,0.003656,0.000037,0.003938,0.009759,0.000097,0.000016,0.073982,0.000214,0.002172,0.000036,0.000807,0.025506,0.000192,0.000322,0.000283,0.010641,2018-02-13 10:39:59,375.0,667.0,36.0,635.0,11,1,US,47.6062,-122.3321
2,152274,0.8,0.4,0.8,0.6,0.6,0.2,0.4,0.6,0.8,0.8,0.8,0.4,0.8,0.4,0.6,0.8,0.6,0.4,0.8,0.8,0.6,0.6,0.8,0.6,0.6,0.4,0.8,0.4,0.6,0.4,0.4,0.8,0.8,0.6,0.4,0.8,0.8,0.4,0.6,...,0.002360,0.007548,0.013643,0.000211,0.000031,0.001508,0.000142,0.000562,0.006754,0.000376,0.004628,0.120100,0.000091,0.009398,0.003656,0.000038,0.000772,0.009781,0.000114,0.000018,0.073979,0.000214,0.002181,0.000037,0.000817,0.025491,0.000170,0.000323,0.000297,0.010646,2016-07-10 23:12:47,320.0,568.0,24.0,298.0,15,1,US,37.7479,-84.2947
3,902438,0.8,0.4,0.8,0.4,0.6,0.2,0.8,0.6,0.8,0.6,0.6,1.0,0.8,0.2,0.4,0.4,0.6,0.2,0.6,0.6,0.2,1.0,0.4,1.0,0.4,0.8,0.4,1.0,1.0,0.8,0.6,0.6,0.8,0.2,0.4,0.4,0.8,0.2,0.8,...,0.002353,0.007528,0.013633,0.000205,0.000030,0.001496,0.000139,0.000561,0.006658,0.000364,0.004628,0.120102,0.000091,0.009398,0.003653,0.000037,0.000760,0.009805,0.000096,0.000027,0.073980,0.000213,0.002162,0.000036,0.000803,0.025477,0.000170,0.000312,0.000283,0.010637,2018-09-07 13:21:56,1920.0,1080.0,252.0,210.0,9,1,US,40.8536,-124.0502
4,597988,0.2,1.0,0.6,1.0,0.2,0.6,0.2,1.0,0.2,1.0,1.0,0.8,1.0,0.6,0.6,1.0,0.6,0.2,0.6,0.6,0.2,0.6,0.2,1.0,0.2,1.0,0.6,0.8,1.0,0.8,0.8,0.2,1.0,0.4,0.6,0.2,0.8,0.4,0.8,...,0.002353,0.007535,0.013641,0.000214,0.000031,0.001502,0.000139,0.000565,0.006700,0.000368,0.004633,0.120106,0.000092,0.009395,0.003665,0.000036,0.000764,0.009764,0.000098,0.000017,0.073984,0.000218,0.002171,0.000036,0.000830,0.025494,0.000172,0.000313,0.000285,0.010644,2018-02-05 12:40:24,375.0,667.0,12.0,245.0,9,1,US,38.8896,-92.2236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304598,600168,0.0,0.2,1.0,0.2,1.0,0.4,1.0,0.4,0.8,0.6,0.4,0.4,0.8,0.4,0.8,0.4,0.6,0.8,0.4,0.4,0.2,0.8,0.4,0.8,0.4,0.8,0.2,0.8,0.8,0.8,1.0,0.2,0.8,0.2,0.8,0.2,1.0,0.6,1.0,...,0.002364,0.007528,0.013648,0.000203,0.000032,0.001497,0.000138,0.000562,0.006661,0.000371,0.004628,0.120104,0.000088,0.009396,0.003656,0.000036,0.000748,0.009761,0.000098,0.000016,0.073980,0.000214,0.002158,0.000036,0.000797,0.025481,0.000165,0.000311,0.000282,0.010639,2018-02-06 16:58:01,1536.0,864.0,15.0,159.0,53,1,US,46.8098,-92.0721
304599,271931,0.6,1.0,0.6,0.4,0.6,0.4,0.6,0.4,0.6,0.8,0.6,0.6,0.8,0.2,0.8,0.6,0.4,

In [9]:
# dropping the empty entries 
datatesting = datatesting.dropna()

In [10]:
# getting rid of unneeded columns 
datatesting = datatesting.drop(['EXT1_E', 'EXT2_E', 'EXT3_E', 'EXT4_E', 'EXT5_E', 'EXT6_E', 'EXT7_E', 'EXT8_E', 'EXT9_E', 'EXT9_E', 'EXT10_E'], axis = 1)
datatesting = datatesting.drop(['EST1_E', 'EST2_E', 'EST3_E', 'EST4_E', 'EST5_E', 'EST6_E', 'EST7_E', 'EST8_E', 'EST9_E', 'EST9_E', 'EST10_E'], axis = 1)
datatesting = datatesting.drop(['AGR1_E', 'AGR2_E', 'AGR3_E', 'AGR4_E', 'AGR5_E', 'AGR6_E', 'AGR7_E', 'AGR8_E', 'AGR9_E', 'AGR9_E', 'AGR10_E'], axis = 1)
datatesting = datatesting.drop(['CSN1_E', 'CSN2_E', 'CSN3_E', 'CSN4_E', 'CSN5_E', 'CSN6_E', 'CSN7_E', 'CSN8_E', 'CSN9_E', 'CSN9_E', 'CSN10_E'], axis = 1)
datatesting = datatesting.drop(['OPN1_E', 'OPN2_E', 'OPN3_E', 'OPN4_E', 'OPN5_E', 'OPN6_E', 'OPN7_E', 'OPN8_E', 'OPN9_E', 'OPN9_E', 'OPN10_E'], axis = 1)
datatesting = datatesting.drop(['dateload', 'screenw', 'screenh', 'introelapse', 'testelapse', 'endelapse', 'IPC', 'country', 'lat_appx_lots_of_err', 'long_appx_lots_of_err'], axis = 1)
datatesting = datatesting.drop(['Unnamed: 0'], axis = 1)

datatesting

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,0.6,0.6,0.6,0.4,0.6,0.4,0.4,0.6,0.6,1.0,0.6,0.8,0.6,0.8,0.4,0.4,0.4,0.4,0.8,0.4,0.2,0.8,0.6,0.8,0.4,1.0,0.4,0.6,0.8,0.8,0.8,0.6,0.8,0.4,0.8,0.4,0.6,0.2,0.8,0.8,0.8,0.2,0.6,1.0,0.8,0.4,0.8,0.8,0.8,0.6
1,0.6,0.4,0.8,0.8,0.8,0.2,0.6,1.0,0.4,0.8,0.4,0.8,0.4,0.4,0.4,0.4,0.8,0.6,0.6,0.8,0.2,1.0,0.6,1.0,0.2,0.8,0.2,0.8,0.8,1.0,0.4,0.8,0.8,0.6,0.4,0.4,0.8,0.8,0.4,0.8,0.6,0.2,0.8,0.2,0.8,0.4,1.0,0.6,0.8,0.8
2,0.8,0.4,0.8,0.6,0.6,0.2,0.4,0.6,0.8,0.8,0.8,0.4,0.8,0.4,0.6,0.8,0.6,0.4,0.8,0.8,0.6,0.6,0.8,0.6,0.6,0.4,0.8,0.4,0.6,0.4,0.4,0.8,0.8,0.6,0.4,0.8,0.8,0.4,0.6,0.8,0.8,0.4,0.8,0.4,0.6,0.4,0.8,0.6,0.8,0.8
3,0.8,0.4,0.8,0.4,0.6,0.2,0.8,0.6,0.8,0.6,0.6,1.0,0.8,0.2,0.4,0.4,0.6,0.2,0.6,0.6,0.2,1.0,0.4,1.0,0.4,0.8,0.4,1.0,1.0,0.8,0.6,0.6,0.8,0.2,0.4,0.4,0.8,0.2,0.8,0.8,0.8,0.2,1.0,0.2,1.0,0.2,0.8,0.4,1.0,0.8
4,0.2,1.0,0.6,1.0,0.2,0.6,0.2,1.0,0.2,1.0,1.0,0.8,1.0,0.6,0.6,1.0,0.6,0.2,0.6,0.6,0.2,0.6,0.2,1.0,0.2,1.0,0.6,0.8,1.0,0.8,0.8,0.2,1.0,0.4,0.6,0.2,0.8,0.4,0.8,0.8,1.0,0.4,1.0,0.4,0.8,0.4,0.8,0.8,0.8,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304598,0.0,0.2,1.0,0.2,1.0,0.4,1.0,0.4,0.8,0.6,0.4,0.4,0.8,0.4,0.8,0.4,0.6,0.8,0.4,0.4,0.2,0.8,0.4,0.8,0.4,0.8,0.2,0.8,0.8,0.8,1.0,0.2,0.8,0.2,0.8,0.2,1.0,0.6,1.0,0.8,0.6,0.8,0.8,0.6,1.0,0.2,0.8,0.8,0.8,0.8
304599,0.6,1.0,0.6,0.4,0.6,0.4,0.6,0.4,0.6,0.8,0.6,0.6,0.8,0.2,0.8,0.6,0.4,0.2,0.4,0.2,0.4,0.6,0.8,0.8,0.0,0.6,0.4,0.8,0.8,0.8,1.0,0.2,1.0,0.2,0.4,0.2,1.0,0.4,1.0,1.0,0.8,0.4,1.0,0.2,0.8,1.0,1.0,0.8,0.8,0.8
304600,0.4,0.8,0.6,0.8,0.6,0.4,0.2,0.8,0.6,0.8,0.6,0.8,0.8,0.4,0.8,0.4,0.4,0.4,0.4,0.8,0.4,1.0,0.4,0.8,0.4,0.8,0.4,1.0,0.8,0.8,0.8,0.8,0.4,0.6,0.8,0.8,0.8,0.4,0.4,0.8,1.0,0.2,1.0,0.4,0.8,0.2,0.8,0.8,0.8,1.0
304601,0.6,0.2,0.6,0.2,0.6,0.8,0.8,0.4,1.0,0.8,0.2,1.0,0.4,0.8,0.4,0.4,0.2,0.2,0.4,0.2,0.2,0.6,0.6,1.0,0.4,1.0,0.4,1.0,1.0,1.0,0.2,0.8,1.0,0.6,0.2,0.6,0.8,0.4,0.2,0.8,0.8,0.2,1.0,0.4,0.8,0.2,1.0,0.6,0.8,1.0


In [11]:
# converting to float
datatraining = datatraining.astype('float')

In [12]:
# putting data in a matrix 

datatesting = np.asarray(datatesting)
datatesting.shape

(303619, 50)

In [13]:
# converting array to a tensor
datatraining = tf.convert_to_tensor(datatraining, dtype=np.float32)
datatesting = tf.convert_to_tensor(datatesting, dtype=np.float32)

In [14]:
# this is to make the shape fit correctly
datatraining = datatraining - tf.expand_dims(tf.reduce_mean(datatraining, axis=1), -1)
datatesting = datatesting - tf.expand_dims(tf.reduce_mean(datatesting, axis=1), -1)

In [15]:
# sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [16]:
# encoder

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
traits = 5
questions = 10
n = 50
latent_dim=2

# using sequential 
encoder = keras.models.Sequential([
    # fully connected layer with 300 nodes, takes in 50 questions
    keras.layers.Dense(300, input_shape=(50, 1), activation="relu"),
    keras.layers.Dropout(rate=0.2),
    # another fully connected layer with 100 nodes
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    # another fully connected layer with 500 nodes
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    # another fully connected layer with 3 nodes 
    keras.layers.Dense(3, activation="softmax")
])

In [17]:
# decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(50 * 1, activation="relu")(latent_inputs)
x = layers.Reshape((50, 1))(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
decoder_outputs = layers.Dense(1, activation="sigmoid")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                150       
_________________________________________________________________
reshape (Reshape)            (None, 50, 1)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 50, 64)            128       
_________________________________________________________________
dense_6 (Dense)              (None, 50, 32)            2080      
_________________________________________________________________
dense_7 (Dense)              (None, 50, 1)             33        
Total params: 2,391
Trainable params: 2,391
Non-trainable params: 0
_________________________________________________________

In [18]:
# defining the vae as a model with a custom train_step
class VAE(keras.Model):
    def __init__(self, encoder, decoder, *args):
        super(VAE, self).__init__(*args)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [19]:
# training the VAE
# the error I am getting is that there are too many values to unpack, I am currently working on the solution

tf.reshape(datatraining, [35421550, 1]).numpy()
bigFive = np.concatenate([datatraining, datatesting], axis=0)
tf.reshape(bigFive, [-1])

bigFive = np.expand_dims(bigFive, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(), run_eagerly=True)
vae.fit(bigFive, epochs=30, batch_size=128)

Epoch 1/30


ValueError: ignored

# ***Scratch code - will probably be deleted later***

In [20]:
#datatraining = tf.autograph.to_graph(datatraining)
#datatesting = tf.autograph.to_graph(datatesting)

#tf.autograph.to_code(datatraining.python_function)

In [32]:
#datatraining = np.reshape(datatraining, (50, 1))

ValueError: ignored

In [ ]:
#datatraining = tf.map_fn(datatraining)
#datatraining = tf.autograph.to_graph(f.datatraining)

#datatraining = to_graph(f.datatraining)

#datatraining = to_graph(datatraining)

datatraining = np.expand_dims(datatraining, -1).astype("float32")
encoder_model = keras.models.Model(encoder_inputs, ([z_mean, z_log_var, latent_z]), name="encoder")'''

vae = VAE(encoder, decoder)

vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(datatraining, epochs=30, batch_size=128)

In [ ]:
datatraining[1] = datatraining[1].str.rstrip('NONE')

KeyError: ignored